# GPT를 활용해 리뷰요약 best prompt 작성
- 우리 서비스에서 사용할 GPT모델 : gpt3.5-turbo

In [10]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 절대경로나 정확한 상대경로 지정
load_dotenv('C:/Users/Admin/kpmg_future_lab/LLM_ex/yanolja_summary/.env', override=True)

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
print(f"API Key loaded: {OPENAI_API_KEY[:10]}...")  # 앞 10글자만 출력 (확인용)

# OpenAI 클라이언트 생성
client = OpenAI(api_key=OPENAI_API_KEY)

# Chat API 호출
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

# 응답 출력
print(response.choices[0].message.content)


API Key loaded: sk-proj-vm...
Hello! How can I assist you today?


# 데이터 전처리 
- 리뷰 데이터 파일 로딩 
- 리뷰 요약 기준 (평점 1-5)
    - good : 평점 5
    - bad : 평점 4이하 
- 각 리뷰마다 스페셜 토큰 
    - '[REVIEW_START]' + r['review'] + '[REVIEW_END]'


## 데이터 전처리 함수 작성 

In [14]:
import json

with open('./res/3.kyongpo_the_hongc.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다',
  'stars': 5,
  'date': '2025.02.03'},
 {'review': '로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다',
  'stars': 5,
  'date': '2024.12.23'},
 {'review': '친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요',
  'stars': 5,
  'date': '2024.12.09'}]

### 좋은 평점 및 나쁜 평점 정의 코드

In [15]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(329, 91)

### 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장 

In [16]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        # specital token 추가
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        # specital token 추가
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_good[:3]

['[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]',
 '[REVIEW_START]로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다[REVIEW_END]',
 '[REVIEW_START]친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요[REVIEW_END]']

### 하나의 문자열로 합치기 

In [17]:
# good 문자, 하나의 문자열로 합치기
reviews_good_text = '\n'.join(reviews_good)
reviews_good_text[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

In [18]:
# bad 문자, 하나의 문자열로 합치기
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text[:100]

'[REVIEW_START]방음이 좀 안되긴 한데 욕실 깨끗해서 좋아요[REVIEW_END]\n[REVIEW_START]공사한다고 예약힐때부터 알려주면좋겠고 체크인할때 공사한다고 불편'

- 하나의 숙박시설에 대한 리뷰 데이터 전처리 함수 정의 

In [ ]:
# 정의부 
import json
import datetime
from dateutil import parser

def preprocess_reviews(data_path):

    with open(data_path, 'r', encoding='utf-8') as f:
        # list 형태로 json 파일 열기
        review_list = json.load(f)
    
    # good, bad 데이터 저장 변수 초기화
    reviews_good, reviews_bad = [], []
    # 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)

    # 리뷰데이터 
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date
        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue
        # 평점 5점 기준
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 하나의 문자열 리스트로 합치기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)# good 리뷰와 bad 리뷰 반환
    return reviews_good_text, reviews_bad_text

In [ ]:
# 호출부 
data_path = "res/1.gangneung_mare.json"
preprocess_reviews(data_path)
# print(preprocess_reviews(data_path))


('[REVIEW_START]바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오는것도 좋았어요~ 친구들이랑 왔다가 다들 만족하고 갑니다ㅎ[REVIEW_END]\n[REVIEW_START]오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 온거였는데 진짜 색달랐어요. 옆에 바다보면서 닭강정먹고.. 또 새벽에 바다 구경하고.. 진짜 잘 보내다 가요🥰[REVIEW_END]\n[REVIEW_START]황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으로 보이는 바다가 너무 아름다웠어요\n직원분들도 모두 친절하셔서 기분 좋게 머물다 갑니다👏\n다음에도 꼭 다시 방문하고 싶어요![REVIEW_END]\n[REVIEW_START]위치좋고 직원들 전부 친절하고 재방문의사100%입니다[REVIEW_END]\n[REVIEW_START]좋았습니다 수고하세요[REVIEW_END]\n[REVIEW_START]괜찬아요\n바다바로앞이라좋아요[REVIEW_END]\n[REVIEW_START]강원도를 좋아하는 여행자입니다.\n강릉 스카이베이,속초 카시아,롯데 다이용해봤지만\n진짜 이가격에 이정도 퀄리티숙소 너무너무 만족하며 이용했습니다\n침구며 객실관리가 청결하게 잘 관리되어있고\n침대도 푹신하니 좋았어요\n단점이라고 하면 이층이라그런가 밤늦게까지 폭죽소리가 너무 시끄러웠고\n비게가 저한테는 높아서 좀 불편했다는 정도..?\n이가격에 오션뷰에 호텔급 침구에 너무 만족했습니다\n재방문의사 100프로입니다[REVIEW_END]\n[REVIEW_START]강문해변 바로 앞이라 위치도 좋고 버스 정류장도 바로 근처라 이동이 너무 편리해서 좋았습니다. 객실과 화장실도 매우 청결 했어요! 너무 쾌적하게 잘 지내다 갑니다[REVIEW_END]\n[REVIEW_START]내부가 깔끔했어요 \n전망도 해안뷰가 이뻤어요\n담에 또 오고싶네요[REVIEW_END]\n[REVIEW_START]잘쉬었어요,,,,,[REV

In [ ]:
# 날짜 정리 
# import datetime
# current_date = datetime.datetime.now()
# current_date - datetime.timedelta(days=6*30)

datetime.datetime(2024, 12, 6, 12, 34, 6, 11136)